# Lab: Cassandra CRUD Operations

![https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-WD0231EN-SkillsNetwork/IDSN-logo.png](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-WD0231EN-SkillsNetwork/IDSN-logo.png)

Estimated time needed: **15** minutes

## **Objectives**

After completing this lab, you will be able to:

- Insert data into a table with an INSERT command
- Read data from a table by querying from it
- Update and delete data from the table based on specific criteria

## Open Docker



In [1]:
!open /Applications/Docker.app

In [2]:
!docker version

Client:
 Cloud integration: v1.0.31
 Version:           23.0.5
 API version:       1.42
 Go version:        go1.19.8
 Git commit:        bc4487a
 Built:             Wed Apr 26 16:12:52 2023
 OS/Arch:           darwin/arm64
 Context:           default

Server: Docker Desktop 4.19.0 (106363)
 Engine:
  Version:          23.0.5
  API version:      1.42 (minimum version 1.12)
  Go version:       go1.19.8
  Git commit:       94d3ad6
  Built:            Wed Apr 26 16:17:14 2023
  OS/Arch:          linux/arm64
  Experimental:     false
 containerd:
  Version:          1.6.20
  GitCommit:        2806fc1057397dbaeefbea0e4e17bddfbd388f38
 runc:
  Version:          1.1.5
  GitCommit:        v1.1.5-0-gf19387a
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0




## **Deploying your Cassandra cluster and running commands**

To deploy your Cassandra cluster, use the Docker CLI in the same folder as your docker-compose.yml to run the following command (the -d causes the containers to run in the background):



In [5]:
!docker compose up -d

[+] Running 0/0
 ⠋ Container cassandra1  Creating                                          0.0s 
[+] Running 1/1
 ✔ Container cassandra1  Created                                           0.1s 
 ⠋ Container cassandra2  Creating                                          0.0s 
[+] Running 3/2
 ✔ Container cassandra1  Created                                           0.1s 
 ✔ Container cassandra2  Created                                           0.1s 
 ✔ Container cassandra3  Created                                           0.1s 
[+] Running 2/3
 ⠿ Container cassandra1  Starting                                          0.3s 
 ✔ Container cassandra2  Created                                           0.1s 
 ✔ Container cassandra3  Created                                           0.1s 
[+] Running 2/3
 ⠿ Container cassandra1  Starting                                          0.4s 
 ✔ Container cassandra2  Created                                           0.1s 
 ✔ Container cassandra3  Crea



To access your Cassandra cluster, you can use csqlsh to connect to the container database using the following commands:





You can also check the cluster configuration using:



In [10]:
!docker exec -it cassandra1 nodetool status



# **Exercise 1 - Create Keyspace and Table**

## **Create `training` keyspace**

Create a keyspace named `training` using `SimpleStrategy` and `replication factor` of `3`.

- Click here for Hint

> use CREATE KEYSPACEcommand with appropriate options
>
- Click here for Solution

On the cqlsh run the below command.



In [9]:
!docker exec -it cassandra1 cqlsh -e "CREATE KEYSPACE training WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 2};"

Connection error: ('Unable to connect to any servers', {'127.0.0.1:9042': ConnectionRefusedError(111, "Tried connecting to [('127.0.0.1', 9042)]. Last error: Connection refused")})


## **Install `Python` Client**

In [ ]:
%pip install cassandra-driver

## **Connect to Cassandra**
Below is a Python script that connects to the Cassandra cluster and selects the training keyspace

In [ ]:
from cassandra.cluster import Cluster

# List of IPs of the Cassandra nodes (obtained previously)
cassandra_ips = ['172.18.0.2', '172.18.0.3', '172.18.0.4']

# Connect to the Cassandra cluster
cluster = Cluster(cassandra_ips)
session = cluster.connect()

# Select the keyspace
session.set_keyspace('training')




## **Create `movies` table**

Create a table named `movies` with columns:

- `movie_id` is an integer and is the primary key.
- `movie_name` is a text column.
- `year_of_release` is an integer.
- Click here for Hint

> use CREATE TABLE command with appropriate options
>
- Click here for Solution

On the cqlsh run the below command.






# **Exercise 2 - Insert data into a table**

Let's insert a row into the table movies.

On cqlsh run the below command.





Verify that the data is saved.





Insert the below movies into the table.

| movie_id | movie_name | year_of_release |
| --- | --- | --- |
| 2 | Jumanji | 1995 |
| 3 | Heat | 1995 |
| 4 | Scream | 1995 |
| 5 | Fargo | 1996 |





# **Exercise 3 - Read data from a table**

In the previous exercise you have inserted some data into the table named `movies`.

Let's query the data in the `movies` table.

Query all rows.





Query the movie name where `movie_id` is `1`.





# **Exercise 4 - Update data in a table**

The `movie_id` for Scream is `4`. It was released in 1996 and not 1995.

Here is how you modify it.





Verify that the update was successful.





# **Exercise 5 - Delete data from a table**

Delete the movie with the movie_id 5.





Verify using the below command.





# **Practice exercises**

1. Problem: Insert the below movie into the movies table.


| movie_id | movie_name | year_of_release |
| --- | --- | --- |
| 6 | Twister | 1997 |

> use the command insert into table’
>





1. Problem: Modify the release year of Twister to 1996.

> use the command ‘update table set column=value where column=value’
>

On the cqlsh run the below command.





1. Problem: Delete the movie Twister.

> use the command delete from ...
>

On the cqlsh run the below command.





1. Problem: Drop the `training` keyspace.

> use the drop keyspace command
>

On the cqlsh run the below command.





# **Summary**

In this lab, you have gained an understanding of CRUD operations in Cassandra.